In [ ]:
!pip install optuna


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.9/78.9 kB 8.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Load dataset
splits = {
    'train': 'Personality Datasets - Reddit/train_set.csv',
    'validation': 'Personality Datasets - Reddit/val_set.csv',
    'test': 'Personality Datasets - Reddit/eval_set.csv'
}
df = pd.read_csv("hf://datasets/Fatima0923/Automated-Personality-Prediction/" + splits["train"])

# Display dataset structure
print(df.head())

# Assume `text` is the column for input text and the following columns are for personality traits
traits = ['agreeableness', 'openness', 'conscientiousness', 'extraversion', 'neuroticism']


# Clean text
def clean_text(text):
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)  # Remove URLs
    text = re.sub(r'\@\w+|\#', '', text)  # Remove mentions and hashtags
    text = re.sub(r'[^A-Za-z\s]', '', text)  # Remove special characters
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with single space
    return text.strip().lower()

df['cleaned_text'] = df['text'].apply(clean_text)

# Normalize trait scores to the range [0, 1]
scaler = MinMaxScaler()
df[traits] = scaler.fit_transform(df[traits])


# Split into train, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(
    df['cleaned_text'], df[traits].values, test_size=0.2, random_state=42
)


                                                text  agreeableness  openness  \
0  his name was kim kimble originally wow thats s...            9.0      61.0   
1  theyre better than the normal posts on ryugioh...           50.0      85.0   
2  how the fuck does this even happen hi youre cu...           15.0      85.0   
3  it probably does ive learned a lot about mysel...           71.0      53.0   
4           yea those are the same sound to me still           64.0      44.0   

   conscientiousness  extraversion  neuroticism  
0               13.0           4.0         72.0  
1               50.0          85.0         50.0  
2               15.0          85.0         15.0  
3               17.0           3.0         31.0  
4               33.0           8.0         88.0  


In [ ]:
from transformers import BertTokenizer
import torch
from torch.utils.data import Dataset, DataLoader

# Load tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenization function
def tokenize(texts, tokenizer, max_length=512):
    return tokenizer(
        texts.tolist(),
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )

# Tokenize training and test sets
train_encodings = tokenize(X_train, tokenizer)
test_encodings = tokenize(X_test, tokenizer)

# Convert labels to tensors
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Create PyTorch Dataset
class PersonalityDataset(Dataset):
    def __init__(self, encodings, targets):
        self.encodings = encodings
        self.targets = targets

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.targets[idx]
        return item

    def __len__(self):
        return len(self.targets)

# Create Dataset instances
train_dataset = PersonalityDataset(train_encodings, y_train_tensor)
test_dataset = PersonalityDataset(test_encodings, y_test_tensor)

# Create DataLoader instances
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
from transformers import BertForSequenceClassification, AdamW

# Initialize the model with 5 outputs for regression
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)

# Move model to GPU if available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Define optimizer
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from torch.nn import MSELoss

# Define loss function
loss_fn = MSELoss()

# Training loop
epochs = 3
for epoch in range(epochs):
    model.train()
    total_train_loss = 0

    for batch in train_loader:
        # Move batch to GPU
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = loss_fn(outputs.logits, labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Accumulate training loss
        total_train_loss += loss.item()

    avg_train_loss = total_train_loss / len(train_loader)
    print(f"Epoch {epoch+1}, Training Loss: {avg_train_loss:.4f}")


Epoch 1, Training Loss: 0.0856
Epoch 2, Training Loss: 0.0767
Epoch 3, Training Loss: 0.0683


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Evaluation loop
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in test_loader:
        # Move batch to GPU
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        preds = outputs.logits.cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(labels.cpu().numpy())

# Calculate evaluation metrics
mae = mean_absolute_error(all_labels, all_preds)
mse = mean_squared_error(all_labels, all_preds)

print(f"Test MAE: {mae:.4f}")
print(f"Test MSE: {mse:.4f}")


Test MAE: 0.2308
Test MSE: 0.0781


# **Stop**

In [ ]:
import optuna
from optuna.storages import RDBStorage
from transformers import BertForSequenceClassification, AdamW
from torch.nn import MSELoss

# Save the study to an SQLite database
storage = RDBStorage("sqlite:///optuna_study.db")  # SQLite database file
study = optuna.create_study(direction='minimize', storage=storage, load_if_exists=True)

def objective(trial):
    # Define hyperparameters to optimize
    learning_rate = trial.suggest_float('learning_rate', 1e-6, 1e-4, log=True)
    batch_size = trial.suggest_categorical('batch_size', [8, 16])
    epochs = trial.suggest_int('epochs', 2, 5)
    eps = trial.suggest_float('eps', 1e-9, 1e-6, log=True)

    # Create DataLoaders with the suggested batch size
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Initialize model
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)
    model.to(device)

    # Optimizer and loss function
    optimizer = AdamW(model.parameters(), lr=learning_rate, eps=eps)
    loss_fn = MSELoss()

    # Training loop
    for epoch in range(epochs):
        model.train()
        total_train_loss = 0

        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # Forward pass
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            loss = loss_fn(outputs.logits, labels)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_train_loss += loss.item()

    # Evaluation
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            preds = outputs.logits.cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())

    # Calculate evaluation metric (MSE in this case)
    mse = mean_squared_error(all_labels, all_preds)
    return mse

# Optimize the hyperparameters
study.optimize(objective, n_trials=20)

# Best hyperparameters
print("Best trial:")
print(f"  Value: {study.best_trial.value}")
print(f"  Params: {study.best_trial.params}")


[I 2024-11-24 02:16:14,824] A new study created in RDB with name: no-name-193798fb-7909-48eb-beed-3c7d11aa990f
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
[I 2024-11-24 02:38:16,164] Trial 0 finished with value: 0.09058496356010437 and parameters: {'learning_rate': 2.7735760982260734e-05, 'batch_size': 8, 'epochs': 5, 'eps': 1.1510259530645845e-07}. Best is trial 0 with value: 0.09058496356010437.
Some weights of BertForSequenceClassifica

KeyboardInterrupt: 

Best Parameters
From the logs:

- learning_rate: 3.858596916921089e-06
- batch_size: 16
- epochs: 5
- eps: 8.656566725261303e-07

In [ ]:
from transformers import BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader
from torch.nn import MSELoss
import torch

# Set device (use GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the best parameters
best_params = {
    'learning_rate': 3.858596916921089e-06,
    'batch_size': 16,
    'epochs': 5,
    'eps': 8.656566725261303e-07
}

# Initialize the model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)
model.to(device)

# Optimizer
optimizer = AdamW(model.parameters(), lr=best_params['learning_rate'], eps=best_params['eps'])

# Loss function
loss_fn = MSELoss()

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=best_params['batch_size'], shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=best_params['batch_size'], shuffle=False)

# Training loop
for epoch in range(best_params['epochs']):
    print(f"Epoch {epoch + 1}/{best_params['epochs']}")

    # Training phase
    model.train()
    total_train_loss = 0
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = loss_fn(outputs.logits, labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()

    print(f"Training loss: {total_train_loss / len(train_loader)}")

# Evaluation phase
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        preds = outputs.logits.cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(labels.cpu().numpy())

# Calculate MSE
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(all_labels, all_preds)
print(f"Test MSE: {mse}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/5
Training loss: 0.09482650138242901
Epoch 2/5
Training loss: 0.08262575688687238
Epoch 3/5
Training loss: 0.07916918258701137
Epoch 4/5
Training loss: 0.0767371526607318
Epoch 5/5
Training loss: 0.07388226666103859
Test MSE: 0.08075298368930817


Key Insights
Model Fit:

The close values of training loss (0.0739) and test MSE (0.0808) suggest that the model does not suffer from significant overfitting or underfitting. The model generalizes reasonably well.
Performance:

While the metrics are close, the overall test MSE of 0.0808 may be relatively high depending on the task or the scale of the target variable. This could indicate room for improvement in accuracy.

In [ ]:
import pandas as pd
founder_tweets_df = pd.read_csv(r"/content/drive/MyDrive/Ordnung/Nova/Cources/3. Semester/Master Thesis/Data_text_variables/Personality/Founders_tweets.csv")
founder_tweets_df

,Unnamed: 0,URL,First Line,Second Line
0,0,https://twitter.com/prpayne5,"Nov 12, 2023",Congratulations to my good friend and colleague
1,1,https://twitter.com/prpayne5,"Oct 23, 2023",@AMIAinformatics
2,2,https://twitter.com/brandonsowers,"Jun 23, 2014",Are your friends using babblr? Find out with o...
3,3,https://twitter.com/brandonsowers,"Jun 17, 2014",The glut of chat apps grows. With ChatCenter y...
4,4,https://twitter.com/brandonsowers,"Jun 12, 2014",Doing some mobile ads? Here are the 3 phases o...
...,...,...,...,...
18306,1343,https://twitter.com/janschwarz,"Oct 28, 2011",http://
18307,1344,https://twitter.com/janschwarz,"Oct 4, 2011",We’re excited to attend our first
18308,1345,https://twitter.com/janschwarz,"Jan 8, 2009",At CES
18309,1346,https://twitter.com/laurenlake1,"Jan 7, 2015",Proud to see


In [ ]:
founder_tweets_df.columns

Index(['Unnamed: 0', 'URL', 'First Line', 'Second Line'], dtype='object')

In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.cuda.amp import autocast

# Define the device (use GPU if available, fallback to CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)
model.to(device)

# Enable gradient checkpointing to reduce memory usage
model.gradient_checkpointing_enable()

# Load the dataframe
# Assuming `founder_tweets_df` contains a column 'Second Line' with the tweets
# Replace this with your actual dataframe loading method if needed

# Function to tokenize text
def tokenize_texts(texts, tokenizer, max_length=128):  # Reduced max_length for memory optimization
    return tokenizer(
        texts.tolist(),
        padding='max_length',
        truncation=True,
        max_length=max_length,
        return_tensors='pt'
    )

# Tokenize and process tweets in batches
batch_size = 4  # Small batch size to reduce memory usage
tweet_texts = founder_tweets_df['Second Line']
personality_scores = []

model.eval()  # Set model to evaluation mode

# Process tweets in smaller batches
with torch.no_grad():
    for i in range(0, len(tweet_texts), batch_size):
        # Get the batch of tweets
        batch_texts = tweet_texts[i:i + batch_size]

        # Tokenize the batch
        encodings = tokenize_texts(batch_texts, tokenizer)

        # Move input tensors to the appropriate device
        input_ids = encodings['input_ids'].to(device)
        attention_mask = encodings['attention_mask'].to(device)

        # Perform inference with mixed precision
        with autocast():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)

        # Extract predictions and move to CPU
        preds = outputs.logits.cpu().numpy()
        personality_scores.extend(preds)

# Convert predictions to a NumPy array
personality_scores = np.array(personality_scores)

# Add the personality scores as new columns in the dataframe
personality_columns = ['agreeableness', 'openness', 'conscientiousness', 'extraversion', 'neuroticism']
for i, column in enumerate(personality_columns):
    founder_tweets_df[column] = personality_scores[:, i]

# Save the updated dataframe
output_file = "founder_tweets_with_personality_updated.csv"
print(f"Predictions added and saved to '{output_file}'")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-7-2f50cd162e12>:53: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Predictions added and saved to 'founder_tweets_with_personality_updated.csv'


In [ ]:
Founder_Twitter_Personality = founder_tweets_df[["URL", "agreeableness", "openness", "conscientiousness", "extraversion", "neuroticism"]].groupby("URL").mean()
Founder_Twitter_Personality

,agreeableness,openness,conscientiousness,extraversion,neuroticism
URL,,,,,
http://twitter.com/204NoContent,0.061360,0.223669,-0.010728,0.193230,-0.230375
http://twitter.com/8globaltech,0.116780,0.195843,0.012749,0.228554,-0.247200
http://twitter.com/AMahadevia,0.118835,0.274048,0.054779,0.236450,-0.245667
http://twitter.com/AbbySturges,0.143147,0.298779,0.072904,0.159606,-0.273096
http://twitter.com/AdamKKaufman,0.072479,0.222575,-0.011078,0.216878,-0.229329
...,...,...,...,...,...
https://www.twitter.com/japantravelcom,0.169312,0.267334,0.114197,0.213135,-0.201538
https://www.twitter.com/pbarnert,0.007904,0.198853,-0.068733,0.206787,-0.274658
https://www.twitter.com/praneethpatlola,0.101187,0.280182,-0.021342,0.250198,-0.228592


In [ ]:
Founder_Twitter_Personality.to_csv(r"/content/drive/MyDrive/Ordnung/Nova/Cources/3. Semester/Master Thesis/Data_text_variables/Personality/Founder_Twitter_Personality.csv")